In [ ]:
%pylab inline
from parcels import FieldSet, Field, ParticleSet, JITParticle, AdvectionRK4, ErrorCode, Variable
import cartopy
from glob import glob
import matplotlib.pyplot as plt
from matplotlib import colors
import numpy as np
import xarray as xr
from netCDF4 import Dataset
import math as math
from ParticlePlotFunctions import *
%matplotlib inline

In [ ]:
# plot SST + quiver of specific date

ddir = '/projects/0/topios/hydrodynamic_data/CMEMS/GLOBAL_REANALYSIS_PHY_001_030/'
datafiles = sorted(glob(ddir+'mercatorglorys12v1_gl12_mean_2008*.nc'))

t=0
latdim = np.arange(800,1100)
londim = np.arange(700,1500)
dfile = Dataset(datafiles[t])
lon = dfile.variables['longitude'][londim]
lat = dfile.variables['latitude'][latdim]
Uvel = dfile.variables['uo'][t,0,latdim,londim]
Vvel = dfile.variables['vo'][t,0,latdim,londim]
Tvel = dfile.variables['thetao'][t,0,latdim,londim]

PlotSSTquiver(lon,lat,Tvel,Uvel,Vvel)

In [ ]:
# compute year mean

ddir = '/projects/0/topios/hydrodynamic_data/CMEMS/GLOBAL_REANALYSIS_PHY_001_030/'
datafiles = sorted(glob(ddir+'mercatorglorys12v1_gl12_mean_2008*.nc'))

latdim = np.arange(800,1100)
londim = np.arange(700,1500)
dfile = Dataset(datafiles[0])
lon = dfile.variables['longitude'][londim]
lat = dfile.variables['latitude'][latdim]

ddates = range(0,365)

Umean = np.zeros([len(lat),len(lon)], dtype=float)
Vmean = np.zeros([len(lat),len(lon)], dtype=float)
Tmean = np.zeros([len(lat),len(lon)], dtype=float)

teller = 0
for t in ddates:
    teller += 1
    print(teller)
    dfile = Dataset(datafiles[t])
    Umean += dfile.variables['uo'][0,0,latdim,londim]
    Vmean += dfile.variables['vo'][0,0,latdim,londim]
    Tmean += dfile.variables['thetao'][0,0,latdim,londim]

Umean = Umean/teller
Vmean = Vmean/teller
Tmean = Tmean/teller

In [ ]:
map_extent = [-105,-75,-10,10]
Umean[Umean<=-100]=nan
Vmean[Vmean<=-100]=nan
Tmean[Tmean<=-100]=nan
PlotSSTquiver(lon,lat,Tmean,Umean,Vmean,figsize=(9,6),map_extent=map_extent)
plt.savefig('../../results/figures/CMEMSfields_mean2008.pdf')

In [ ]:
figsize=(9,6)
Tmin=-0.5
Tmax=0.5
fig, ax = plt.subplots(figsize=figsize)

levels = np.linspace(Tmin, Tmax, 41)
fig = ax.contourf(lon,lat,Vmean,       
                levels = levels,    
                cmap='Spectral_r',       
                extend='both',      
                origin='lower')     
ax.set_title('sea surface temperature')          
ax.set_xlabel('longitude')                 
ax.set_ylabel('latitude') 
#ax.set_xlim(map_extent[0:2])
#ax.set_ylim(map_extent[2:4])
ax.set_facecolor('gray')
cbar = plt.colorbar(fig, ax=ax)
cbar.ax.set_ylabel('SST (\xb0C)')